# Relevant Bits of Code for Napari 

## Data Preparation
### Unstacking 

This code snippet helps unstack a stacked tif file with txy format. 

In [ ]:
import imageio
import os

def extract_tiff_frames_with_imageio(tiff_path, output_folder):
    reader = imageio.get_reader(tiff_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i, im in enumerate(reader):
        if im.ndim == 2:  
            imageio.imwrite(f"{output_folder}/frame_{str(i+1).zfill(4)}.tif", im)
        elif im.ndim == 3:  
            for j, slice in enumerate(im):
                imageio.imwrite(f"{output_folder}/frame_{str(i+1).zfill(4)}_slice_{str(j+1).zfill(4)}.tif", slice)
        else:
            print(f"Frame {i+1} has unsupported dimensions: {im.ndim}")

data = # folder
file = # image

extract_tiff_frames_with_imageio(file, data)

## Data Visualization
### Ground Truth vs Channel Info
This code snippet allows loading data in Napari in a way such that it is possible for the user to understand what objects from the ground truth images are missed by two diferent channel settings. 

In [ ]:
import os
import numpy as np
from skimage import io
import napari

%gui qt

mask_dir_1 = # first channel option (folder of unstacked images with this channel setting)
mask_dir_2 = # second channel option (folder of unstacked images with this channel setting)
mask_dir_3 = # folder with ground truths

def load_mask_stack(mask_dir):
    mask_files = sorted([f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))])
    frames = [io.imread(os.path.join(mask_dir, f)) for f in mask_files]
    return np.stack(frames)

def convert_to_color_mask(label_stack, color):
    rgba = np.zeros((*label_stack.shape, 4), dtype=np.uint8) 
    mask = label_stack > 0 
    
    rgba[..., :3] = np.array(color)  
    rgba[..., 3] = mask.astype(np.uint8) * 255 
    return rgba

stacked_mask_1 = load_mask_stack(mask_dir_1)
stacked_mask_2 = load_mask_stack(mask_dir_2)
stacked_mask_3 = load_mask_stack(mask_dir_3)

mask_1_rgba = convert_to_color_mask(stacked_mask_1, [0, 0, 255])  # bluen
mask_2_rgba = convert_to_color_mask(stacked_mask_2, [255, 0, 0])  # red
mask_3_rgba = convert_to_color_mask(stacked_mask_3, [0, 255, 0])  # green

viewer = napari.Viewer()

viewer.add_image(mask_1_rgba, name="[1, 0] - blue", opacity=0.5, blending="additive")
viewer.add_image(mask_2_rgba, name="[1, 2] - red", opacity=0.5, blending="additive")
viewer.add_image(mask_3_rgba, name="Ground truths - green", opacity=0.5, blending="additive")

napari.run()


### Visualization of tracks (optical plus segmentation)
This code snippet allows launching the Napari GUI with layers representing optical, segmentation, and tracking data respectively. 

In [ ]:
import pandas as pd
import numpy as np
import os
from skimage import io
import napari

%gui qt

image_dir = # images directory (unstacked)
mask_dir_1 = # optical unstacked
mask_dir_2 = # cellpose unstacked
data_path = # tracking data (csv file)

image_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
frames = [io.imread(os.path.join(image_dir, image_file)) for image_file in image_files]
stacked_frames = np.stack(frames)
viewer = napari.view_image(stacked_frames)
mask_dir_1_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
frames_mask_dir_1 = [io.imread(os.path.join(mask_dir_1, mask_dir_1_files)) for mask_dir_1_files in mask_dir_1_files]
stacked_frames_mask_dir_1 = np.stack(frames_mask_dir_1)
viewer.add_image(stacked_frames_mask_dir_1)
mask_dir_2_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
frames_mask_dir_2 = [io.imread(os.path.join(mask_dir_2, mask_dir_2_files)) for mask_dir_2_files in mask_dir_2_files]
stacked_frames_mask_dir_2 = np.stack(frames_mask_dir_2)
viewer.add_labels(stacked_frames_mask_dir_2)

dataf = pd.read_csv(data_path, low_memory=False)
datafilter = dataf.copy()
datafilter = datafilter.dropna(subset=['id']) # depends on column names in input data (csv file)
scale_factor = 1
datafilter["SCALED_Y"] = datafilter["posy"] * scale_factor # depends on column names in input data (csv file)
datafilter["SCALED_X"] = datafilter["posx"] * scale_factor # depends on column names in input data (csv file)
track3 = datafilter.reset_index()
viewer.add_tracks(track3[["id", "frame", "SCALED_X", "SCALED_Y"]]) # depends on column names in input data (csv file)
viewer.add_tracks(track3[["id", "frame", "SCALED_X", "SCALED_Y"]]) # depends on column names in input data (csv file)